# Data pre-process

### Imports and paths initialization

In [2]:
# imports for the whole notebook
from xml.etree import ElementTree as ET
import csv
import pandas as pd
import re
import numpy as np
import math
import os
import argparse


def parse_args(args=None):
    parser = argparse.ArgumentParser(
        description='Data pre-process',
        usage=''
    )
    parser.add_argument('--data_dir', type=str, default='data')
    parser.add_argument('--onto_dir', type=str, default='ontology')
    parser.add_argument('--syn_data_dir', type=str, default='syn_data')

    return parser.parse_args(args)


# set the path to your data folders here
args = parse_args(args=['--data_dir', '../persistent/data'])
onto_dir_path = os.path.join(args.data_dir, args.onto_dir)
syn_data_dir_path = os.path.join(args.data_dir, args.syn_data_dir)

if not os.path.exists(onto_dir_path):
    raise ValueError(f'You need an existing ontology directory with the \
                     XML dataset inside it, please create \'{onto_dir_path}\'')

if not os.path.exists(syn_data_dir_path):
    os.makedirs(syn_data_dir_path)

### Convert XML dataset to CSV

In [31]:
# Dataset: clinical signs and symptoms in rare diseases
# http://www.orphadata.org/cgi-bin/index.php (Phenotypes associated with rare disorders)


tree = ET.parse(os.path.join(onto_dir_path, 'en_product4.xml'))
root = tree.getroot()


headers = ['HPODisorderSetStatus_id', 'Disorder_id', 'OrphaCode', 'ExpertLink',
           'Name', 'DisorderType_id', 'DisorderType_name', 'DisorderGroup_id',
           'DisorderGroup_Name', 'HPODisorderAssociation_id', 'HPO_id',
           'HPOId', 'HPOTerm', 'HPOFrequency_id', 'HPOFrequency_Name',
           'DiagnosticCriteria_id', 'DiagnosticCriteria_Name', 'Source',
           'ValidationStatus', 'Online', 'ValidationDate']


def find_value(row_data, source_tag, target_tag_name, field, text=True):
    """Finds a sub-tag of a source tag and inputs its value into a dictionary
        containing the current row's data

    Args:
        row_data (dict):
            The data for the current row associated with the csv fields
        source_tag (Element):
            XML parent tag to search from
        target_tag_name (str):
            Name of the sub-tag to find
        field (str):
            Field in the csv file
        text (bool):
            Indicates if the value of the tag to retrieve is its inner text 
            or its id attribute
    Returns:
        tag (Element):
            Returns the found tag
    """
    tag = source_tag.find(target_tag_name)
    tag_v = ''

    if tag is not None:  # retrieving either the inner text or the id attribute of the tag
        if text:
            tag_v = tag.text
        elif (len(tag.attrib) > 0):
            tag_v = tag.attrib['id']
    row_data[field] = tag_v if tag_v is not None else ''

    return tag


with open(os.path.join(onto_dir_path, 'en_product4.csv'), 'w', encoding='utf-8') as fd:
    csvwriter = csv.DictWriter(fd, delimiter=',', fieldnames=headers)
    csvwriter.writeheader()

    # iterating through all the disorders
    for status in root.find('HPODisorderSetStatusList').findall('HPODisorderSetStatus'):
        row_data = {}
        row_data['HPODisorderSetStatus_id'] = status.attrib['id']

        disorder_tag = find_value(row_data, status, 'Disorder', 'Disorder_id', text=False)
        find_value(row_data, disorder_tag, 'OrphaCode', 'OrphaCode', text=True)
        find_value(row_data, disorder_tag, 'ExpertLink', 'ExpertLink', text=True)
        find_value(row_data, disorder_tag, 'Name', 'Name', text=True)

        disordertype_tag = find_value(row_data, disorder_tag, 'DisorderType', 'DisorderType_id', text=False)
        find_value(row_data, disordertype_tag, 'Name', 'DisorderType_name', text=True)
        disordergroup_tag = find_value(row_data, disorder_tag, 'DisorderGroup', 'DisorderGroup_id', text=False)
        find_value(row_data, disordergroup_tag, 'Name', 'DisorderGroup_Name', text=True)

        for field in ['Source', 'ValidationStatus', 'Online', 'ValidationDate']:
            find_value(row_data, status, field, field, text=True)

        # iterating through all the disorder associations and writing a row for each association
        for association in disorder_tag.find('HPODisorderAssociationList').findall('HPODisorderAssociation'):
            row_data['HPODisorderAssociation_id'] = association.attrib['id']

            hpo_tag = find_value(row_data, association, 'HPO', 'HPO_id', text=False)
            find_value(row_data, hpo_tag, 'HPOId', 'HPOId', text=True)
            find_value(row_data, hpo_tag, 'HPOTerm', 'HPOTerm', text=True)
            hpofrequency_tag = find_value(row_data, association, 'HPOFrequency', 'HPOFrequency_id', text=False)
            find_value(row_data, hpofrequency_tag, 'Name', 'HPOFrequency_Name', text=True)

            diagnosticcriteria_tag = find_value(row_data, association, 'DiagnosticCriteria', 'DiagnosticCriteria_id', text=False)
            find_value(row_data, diagnosticcriteria_tag, 'Name', 'DiagnosticCriteria_Name', text=True)

            csvwriter.writerow(row_data)

In [32]:
df = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'))
df.head(5)

,HPODisorderSetStatus_id,Disorder_id,OrphaCode,ExpertLink,Name,DisorderType_id,DisorderType_name,DisorderGroup_id,DisorderGroup_Name,HPODisorderAssociation_id,...,HPOId,HPOTerm,HPOFrequency_id,HPOFrequency_Name,DiagnosticCriteria_id,DiagnosticCriteria_Name,Source,ValidationStatus,Online,ValidationDate
0,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327485,...,HP:0000256,Macrocephaly,28412,Very frequent (99-80%),NaN,NaN,NaN,y,y,2016-06-01 00:00:00.0
1,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327486,...,HP:0001249,Intellectual disability,28412,Very frequent (99-80%),NaN,NaN,NaN,y,y,2016-06-01 00:00:00.0
2,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327487,...,HP:0001250,Seizures,28412,Very frequent (99-80%),NaN,NaN,NaN,y,y,2016-06-01 00:00:00.0
3,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327488,...,HP:0001257,Spasticity,28412,Very frequent (99-80%),NaN,NaN,NaN,y,y,2016-06-01 00:00:00.0
4,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327489,...,HP:0001274,Agenesis of corpus callosum,28412,Very frequent (99-80%),NaN,NaN,NaN,y,y,2016-06-01 00:00:00.0


### Dataset to triples, entities and relations

In [33]:
freq_assoc = {  # from csv frequency to frequency code
    'Obligate (100%)': 'O',
    'Very frequent (99-80%)': 'VF',
    'Frequent (79-30%)': 'F',
    'Occasional (29-5%)': 'OC',
    'Very rare (<4-1%)': 'VR',
    'Excluded (0%)': 'E'
}

freq_code_assoc = {  # from frequency code to output class
    'O': 'obligate',
    'VF': 'very_frequent',
    'F': 'frequent',
    'OC': 'occasional',
    'VR': 'very_rare',
    'E': 'excluded'
}

dc_association = {  # default: exclusion
    'Diagnostic criterion': 'diagnostic_criterion',
    'Pathognomonic sign': 'pathognomonic_sign',
}


def get_association_subclass(orpha, freq, hp):
    """Returns normalized association class

    Args:
        orpha (str):
            The prefixed Orphanet code
        freq (str):
            The frequency text
        hp (str):
            The prefixed HPO ID
    Returns:
        (str):
            The orphacode, hpo id and frequency association
    """
    return orpha + '_' + hp + '_FREQ:' + freq_assoc.get(freq)


def get_association_name(orpha, freq, hp):
    """Returns textual description of the association class

    Args:
        orpha (str):
            The prefixed Orphanet code
        freq (str):
            The frequency text
        hp (str):
            The prefixed HPO ID
    Returns:
        (str):
            The orphacode, hpo id and frequency association 
            textual_description_with_underscores
    """
    return get_normalized_string(orpha_entities.get(orpha) + ' and ' +
                                 hpo_entities.get(hp) + ' ' +
                                 freq_code_assoc.get(freq_assoc.get(freq)) +
                                 ' association')


def get_normalized_string(s):
    """Transforms a string to lowercase and replaces all whitespace runs with an underscore

    Args:
        s (str):
            String to normalize
    Returns:
        (str):
            The normalized string
    """
    return re.sub(r"\s+", '_', s.lower())


df_dataset = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'), 
                         dtype='object')
df_dataset['OrphaCode'] = df_dataset['OrphaCode'].map(lambda x: 'ORPHA:' + x)

# key is id, value is textual_description_with_underscores
assoc_entities = {}
dc_entities = {'diagnostic_criterion': 'diagnostic_criterion',
               'pathognomonic_sign': 'pathognomonic_sign',
               'exclusion': 'exclusion'}
freq_assoc_entities = {'obligate': 'obligate', 'very_frequent': 'very_frequent',
                       'frequent': 'frequent', 'occasional': 'occasional',
                       'very_rare': 'very_rare', 'excluded': 'excluded'}
hpo_entities = {}
orpha_entities = {}

has_object_triples = []  # association has_object HPOId
has_subject_triples = []  # association has_subject OrphaCode
has_frequency_triples = []  # association has_frequency FrequencyAssociation
has_diagnostic_criterion_triples = []  # association has_DC_attribute DC


# reading the dataset
for orpha, freq, hp, dc, \
    orpha_name, hpo_name in zip(df_dataset['OrphaCode'],
                                df_dataset['HPOFrequency_Name'],
                                df_dataset['HPOId'],
                                df_dataset['DiagnosticCriteria_Name'],
                                df_dataset['Name'],
                                df_dataset['HPOTerm']):
    if hp not in hpo_entities:
        hpo_entities[hp] = get_normalized_string(hpo_name)
    if orpha not in orpha_entities:
        orpha_entities[orpha] = get_normalized_string(orpha_name)

    ac = get_association_subclass(orpha, freq, hp)
    ac_name = get_association_name(orpha, freq, hp)
    assoc_entities[ac] = ac_name

    has_object_triples.append((ac, 'association_has_object', hp))
    has_subject_triples.append((ac, 'association_has_subject', orpha))
    has_frequency_triples.append((ac, 'has_frequency', freq_code_assoc.get(freq_assoc.get(freq))))
    has_diagnostic_criterion_triples.append((ac, 'has_DC_attribute', dc_association.get(dc, 'exclusion')))


# lists corresponding to each output file
triples = []
triples_names = []
entities = []
entities_names = []
relations = []

# subClassOf triples
entities_and_parent_class = [
    (assoc_entities, 'association'),
    (dc_entities, 'diagnostic_criteria'), 
    (freq_assoc_entities, 'frequency_association'), 
    (hpo_entities, 'HPO_Id'), 
    (orpha_entities, 'OrphaCode')
]
for (entities_dict, parent_class) in entities_and_parent_class:
    for k, v in entities_dict.items(): 
        triples.append((k, 'subClassOf', get_normalized_string(parent_class)))
        triples_names.append((v, 'subClassOf', parent_class))

# other properties triples
triples_and_entities = [
    (has_object_triples, hpo_entities), 
    (has_subject_triples, orpha_entities), 
    (has_frequency_triples, freq_assoc_entities), 
    (has_diagnostic_criterion_triples, dc_entities)
]
for (properties_triples, entities_dict) in triples_and_entities:
    for (s, r, o) in properties_triples:
        triples.append((s, r, o))
        triples_names.append((assoc_entities.get(s), r, entities_dict.get(o)))

# parent entities
for i, (k, v) in enumerate(entities_and_parent_class):
    parent = get_normalized_string(v)
    entities.append((i, parent))
    entities_names.append((i, parent))
# entities
parents_count = len(entities)
for i, (k, v) in enumerate({**assoc_entities, **dc_entities, **freq_assoc_entities,
                            **hpo_entities, **orpha_entities}.items()):
    entities.append((i+parents_count, k))
    entities_names.append((i+parents_count, v))

# relations
for i, r in enumerate(['subClassOf', 'association_has_object',
                       'association_has_subject', 'has_frequency', 
                       'has_DC_attribute']):
    relations.append((i, r))


# writing to the different files
lists_and_files = [
    (triples, 'triples.txt'), 
    (triples_names, 'triples_names.txt'),
    (entities, 'entities.dict'),
    (entities_names, 'entities_names.dict'),
    (relations, 'relations.dict')
]
for (l, n) in lists_and_files:
    with open(os.path.join(onto_dir_path, n), 'w') as f:
        for t in l:
            f.write('\t'.join(str(e) for e in t) + '\n')

### Merge ORDO and HP ontologies using the dataset (unused)

In [27]:
df_ordo = pd.read_csv(os.path.join(onto_dir_path, 'ORDO.csv'), dtype='object')
df_hp = pd.read_csv(os.path.join(onto_dir_path, 'HP.csv'), dtype='object')
df_dataset = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'), dtype='object')


# prefixes to distinguish the columns from the 2 ontologies
df_ordo = df_ordo.add_prefix('ORDO_')
df_hp = df_hp.add_prefix('HP_')

# normalizing the different columns for the merge
df_ordo['OrphaCode'] = df_ordo['ORDO_Class ID'].map(lambda x: x.replace('http://www.orpha.net/ORDO/Orphanet_', ''))
df_hp['HPOId'] = df_hp['HP_http://www.w3.org/2004/02/skos/core#notation']
df_dataset['OrphaCode'] = df_dataset['OrphaCode'].astype(str)

# merge
df_merged = pd.merge(df_dataset, df_hp, how='left', on='HPOId')
df_merged = pd.merge(df_merged, df_ordo, how='left', on='OrphaCode')

df_merged.head(1000).to_csv(os.path.join(onto_dir_path, 'merged_ontologies.csv'), encoding='utf-8', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../persistent/data/ontology/ORDO.csv'

In [3]:
df_dataset = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'), dtype='object')
df_res = pd.read_csv(os.path.join(onto_dir_path, 'merged_ontologies.csv'), dtype='object')

df_res.head(5)

,HPODisorderSetStatus_id,Disorder_id,OrphaCode,ExpertLink,Name,DisorderType_id,DisorderType_name,DisorderGroup_id,DisorderGroup_Name,HPODisorderAssociation_id,...,ORDO_http://www.w3.org/2004/02/skos/core#notation,ORDO_https://creativecommons.org/licenses/permits,ORDO_https://creativecommons.org/licenses/requires,ORDO_major susceptibility factor in,ORDO_manual_assertion,ORDO_modifying germline mutation in,ORDO_part of a fusion gene in,ORDO_part_of,ORDO_present_in,ORDO_role in the phenotype of
0,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327485,...,ORPHA:58,NaN,NaN,NaN,NaN,NaN,NaN,http://www.orpha.net/ORDO/Orphanet_182070|http...,http://www.orpha.net/ORDO/Orphanet_410102|http...,NaN
1,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327486,...,ORPHA:58,NaN,NaN,NaN,NaN,NaN,NaN,http://www.orpha.net/ORDO/Orphanet_182070|http...,http://www.orpha.net/ORDO/Orphanet_410102|http...,NaN
2,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327487,...,ORPHA:58,NaN,NaN,NaN,NaN,NaN,NaN,http://www.orpha.net/ORDO/Orphanet_182070|http...,http://www.orpha.net/ORDO/Orphanet_410102|http...,NaN
3,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327488,...,ORPHA:58,NaN,NaN,NaN,NaN,NaN,NaN,http://www.orpha.net/ORDO/Orphanet_182070|http...,http://www.orpha.net/ORDO/Orphanet_410102|http...,NaN
4,1,2,58,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Alexander disease,21394,Disease,36547,Disorder,327489,...,ORPHA:58,NaN,NaN,NaN,NaN,NaN,NaN,http://www.orpha.net/ORDO/Orphanet_182070|http...,http://www.orpha.net/ORDO/Orphanet_410102|http...,NaN


### Synthetic data generation from the ontology

In [14]:
frequency_dict = {  # frequency ids + associated probability
    28405: 1,  # Obligate (100%)
    28412: 0.895,  # Very frequent (99-80%)
    28419: 0.545,  # Frequent (79-30%)
    28426: 0.17,  # Occasional (29-5%)
    28433: 0.025,  # Very rare (<4-1%)
    28440: 0  # Excluded (0%)
}


def get_normalized_string(s):
    """Transforms a string to lowercase and replaces all whitespace runs with an underscore

    Args:
        s (str):
            String to normalize
    Returns:
        (str):
            The normalized string
    """
    return re.sub(r"\s+", '_', s.lower())


def gen_syn_data(patients_per_rd=10, names=True, unseen_pct=0.2, gen_small_file=True):
    """Generates synthetic seen and unseen data from the ontology into files

    Args:
        patients_per_rd (int):
            Number of generated patients per RD
        names (bool):
            Whether the files listing the names of RDs and HPs instead of their codes/ids should be generated too
        unseen_pct (float):
            The percentage of RDs to keep for unseen patients samples
        gen_small_file (bool):
            Whether to generate very small files to debug the model or not
    """
    df = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'))

    # randomizing the order of the RDs for randomized seen/unseen RDs
    df_shuffled = df.sample(frac=1).reset_index(drop=True)
    grouped = df_shuffled.groupby('OrphaCode', sort=False)

    patients_count = 0
    seen_patients_data = []
    seen_patients_data_names = []
    unseen_patients_data = []
    unseen_patients_data_names = []
    lists_n_files = [
        (seen_patients_data, 'syn_patients_data_seen.txt') ,
        (seen_patients_data_names, 'syn_patients_data_seen_names.txt'),
        (unseen_patients_data, 'syn_patients_data_unseen.txt'),
        (unseen_patients_data_names, 'syn_patients_data_unseen_names.txt')
    ]

    rd_count = grouped.ngroups
    seen_unseen_th = rd_count - math.ceil(rd_count*unseen_pct)  # threshold of RDs to switch to unseen RDs

    for group_nb, (name, group) in enumerate(grouped):  # for each RD
        hp_count = len(group)
        # generate patients_per_rd patients for each RD
        for patient_id in range(patients_count, patients_count+patients_per_rd):
            proba_results = np.random.rand(hp_count)  # generating random floats for probabilities
            for i, (orphacode, rd_name, hpo_id, 
                    hp_name, frequency_id) in enumerate(zip(group['OrphaCode'],
                                                            group['Name'],
                                                            group['HPOId'],
                                                            group['HPOTerm'],
                                                            group['HPOFrequency_id'])):
                if (proba_results[i] >= 1 - frequency_dict[frequency_id]):  # comparing generated float and proba
                    if (group_nb <= seen_unseen_th):
                        seen_patients_data.append((patient_id, orphacode, hpo_id))
                        if names:
                            seen_patients_data_names.append((patient_id, get_normalized_string(rd_name), get_normalized_string(hp_name)))
                    else:
                        unseen_patients_data.append((patient_id, hpo_id))
                        if names:
                            unseen_patients_data_names.append((patient_id, get_normalized_string(hp_name)))

        patients_count += patients_per_rd

    # writing the 2 or 4 files
    for lst, fn in lists_n_files:
        if (len(lst) > 0):
            pd.DataFrame(lst).to_csv(
                os.path.join(syn_data_dir_path, fn),
                encoding='utf-8', index=False, header=False
            )

    print(f'Total RDs: {rd_count} seen RDs: {seen_unseen_th}, unseen RDs: {rd_count-seen_unseen_th}')

    if gen_small_file:
        NUMBER_OF_DISEASES = 5
        print('Generating smaller data files for debugging')
        for lst, fn in lists_n_files:
            if (len(lst) > 0):
                temp = []
                nb = i = 0
                while nb <= NUMBER_OF_DISEASES:
                    if lst[i][1] not in temp:
                        temp.append(lst[i][1])
                        nb += 1
                        i -= 1
                    i += 1
                pd.DataFrame(lst[:i]).to_csv(
                    os.path.join(syn_data_dir_path, 'small_' + fn),
                    encoding='utf-8', index=False, header=False
                )
        print(f'Number of diseases: {NUMBER_OF_DISEASES}')


gen_syn_data()

Total RDs: 4243 seen RDs: 3394, unseen RDs: 849
Generating smaller data files for debugging
Number of diseases: 5


### OrphaCode and HPO IDs dictionaries

In [35]:
df = pd.read_csv(os.path.join(onto_dir_path, 'en_product4.csv'))

df_hp = df[['HPOId', 'HPOTerm']].drop_duplicates()
df_rd = df[['OrphaCode', 'Name']].drop_duplicates()
df_rd['OrphaCode'] = 'ORPHA:' + df_rd['OrphaCode'].astype(str)

df_hp.to_csv(os.path.join(onto_dir_path, 'hp.dict'), encoding='utf-8', index=False, header=False)
df_rd.to_csv(os.path.join(onto_dir_path, 'rd.dict'), encoding='utf-8', index=False, header=False)